In [1]:
import xarray as xr
import os
import glob
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List
from pathlib import Path
from datetime import datetime

In [2]:
path = '/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16'

In [3]:
filenames = glob.glob(f'{path}/**', recursive=True)

In [4]:
filenames

['/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/',
 '/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/L1b',
 '/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/L1b/noaa-goes16',
 '/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/L1b/noaa-goes16/ABI-L1b-RadF',
 '/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/L1b/noaa-goes16/ABI-L1b-RadF/2018',
 '/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/L1b/noaa-goes16/ABI-L1b-RadF/2018/274',
 '/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/L1b/noaa-goes16/ABI-L1b-RadF/2018/274/14',
 '/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/L1b/noaa-goes16/ABI-L1b-RadF/2018/274/14/OR_ABI-L1b-RadF-M4C16_G16_s20182741400224_e20182741405040_c20182741405096.nc',
 '/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/L1b/noaa-goes16/ABI-L1b-RadF/2018/274/14/OR_ABI-L1b-RadF-M4C15_G16_s20182741400224_e20182741405034_c20182741405099.nc',
 '/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/L1b/noaa-goes

In [54]:

@dataclass(order=True, frozen=True)
class GOESFileName:
    """
    GOES Data product file name
    noaa-goes[sat-no]/[instrument]-[level]-[product][domain]/[year]/[day]/[hour]/
        OR_[instrument]-[level]-[product][domain]-M4C[channel]_G[sat-no]_s[YYYYDDDHHMMSSS]_e[YYYYDDDHHMMSSS]_c[YYYYDDDHHMMSSS].nc
        s = start date
        e = end date
        c = creation date
    GOES Cloud mask file name
    noaa-goes[sat-no]/[instrument]-[level]-[product][domain]/[year]/[day]/[hour]/
        OR_[instrument]-[level]-[product][domain]-M6_G[sat-no]_s[YYYYDDDHHMMSSS]_e[YYYYDDDHHMMSSS]_c[YYYYDDDHHMMSSS].nc

    """
    save_path: str 
    instrument: str
    processing_level: str
    satellite_number: str
    product: str
    domain: str
    identifier: str
    ext: str
    datetime_acquisition_start: datetime
    datetime_acquisition_end: datetime
    datetime_processing: datetime

    @classmethod
    def from_filename(cls, file_name: str):
        """
        Creates a GOESFileName object from a given file name.

        Args:
            cls (type): The class object that the method is bound to.
            file_name (str): The file name to parse.

        Returns:
            GOESFileName: The parsed GOESFileName object.
        """

        file_name = Path(file_name)
        print(file_name)
        components = file_name.name.split('_')
        save_path = str(file_name.parents[0])

        data_product = components[1]
        data_product_components = data_product.split('-')
        instrument = data_product_components[0]
        processing_level = data_product_components[1]
        product = data_product_components[2][:-1]
        domain = data_product_components[2][-1]
        identifier = data_product_components[3]
        satellite_number = components[2][-2:]

        # acquisition start time and date
        time = components[3][8:]
        year = components[3][1:5]
        day = components[3][5:8]
        datetime_acquisition_start = datetime.strptime(f"{year}{day}{time}", "%Y%j%H%M%S%f")

        # acquisition end time and date
        time = components[4][8:]
        year = components[4][1:5]
        day = components[4][5:8]
        datetime_acquisition_end = datetime.strptime(f"{year}{day}{time}", "%Y%j%H%M%S%f")

        # processing time and date
        time = components[5][8:-3]
        year = components[5][1:5]
        day = components[5][5:8]
        datetime_processing = datetime.strptime(f"{year}{day}{time}", "%Y%j%H%M%S%f")
        
        ext = components[5].split('.')[1]
        return cls(
            save_path=save_path,
            instrument = instrument,
            processing_level = processing_level,
            satellite_number=satellite_number,
            product=product,
            domain=domain,
            identifier=identifier,
            datetime_acquisition_start=datetime_acquisition_start,
            datetime_acquisition_end=datetime_acquisition_end,
            datetime_processing=datetime_processing,
            ext = ext
            )
    
    @property
    def goes_filename(self):
        """
        Generates the GOES file name based on the object's properties.

        Returns:
            str: The generated GOES file name.
        OR_[instrument]-[level]-[product][domain]-M4C[channel]_G[sat-no]_s[YYYYDDDHHMMSSS]_e[YYYYDDDHHMMSSS]_c[YYYYDDDHHMMSSS].nc

        """
        # data product
        filename = f"OR_{self.instrument}-{self.processing_level}-{self.product}{self.domain}-{self.identifier}"
        # satellite number
        filename += f"_G{self.satellite_number}"
        # acquisition start and end
        date_acquisition_start = self.datetime_acquisition_start.strftime("%Y%j%H%M%S%f")
        date_acquisition_end = self.datetime_acquisition_end.strftime("%Y%j%H%M%S%f")
        filename += f"_s{date_acquisition_start}_e{date_acquisition_end}"
        # processing time
        date_processing = self.datetime_processing.strftime("%Y%j%H%M%S%f")
        filename += f"_c{date_processing}"
        # extension
        filename += f".{self.ext}"
        return filename
    
    @property
    def full_path(self):
        """
        Gets the full path of the GOES file.

        Returns:
            Path: The full path of the GOES file.
        """
        return Path(self.save_path).joinpath(self.goes_filename)
    

In [60]:
GOESFileName.from_filename(filenames[10])

/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/L1b/noaa-goes16/ABI-L1b-RadF/2018/274/14/OR_ABI-L1b-RadF-M4C05_G16_s20182741400224_e20182741405027_c20182741405080.nc


GOESFileName(save_path='/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/L1b/noaa-goes16/ABI-L1b-RadF/2018/274/14', instrument='ABI', processing_level='L1b', satellite_number='16', product='Rad', domain='F', identifier='M4C05', ext='nc', datetime_acquisition_start=datetime.datetime(2018, 10, 1, 14, 0, 22, 400000), datetime_acquisition_end=datetime.datetime(2018, 10, 1, 14, 5, 2, 700000), datetime_processing=datetime.datetime(2018, 10, 1, 14, 5, 8))

In [4]:
ds = xr.open_dataset(filenames[10])

In [9]:
ds.band_wavelength

<xarray.DataArray 'band_wavelength' (band: 1)>
[1 values with dtype=float32]
Coordinates:
    t                datetime64[ns] ...
    y_image          float32 ...
    x_image          float32 ...
    band_id          (band) int8 ...
    band_wavelength  (band) float32 ...
Dimensions without coordinates: band
Attributes:
    long_name:      ABI band central wavelength
    standard_name:  sensor_band_central_radiation_wavelength
    units:          um

In [10]:
from rs_tools._src.utils.io import get_list_filenames


/opt/anaconda3/envs/iti-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
def parse_goes16_dates_from_file(file: str):
    timestamp = Path(file).name.replace("-","_").split("_")
    return datetime.strptime(timestamp[-2][1:], "%Y%j%H%M%S%f").strftime("%Y%j%H%M")


In [12]:
def goes_files(read_path):
        """
        Returns a list of all GOES files in the read path.

        Returns:
            List[str]: A list of file paths.
        """
        # get a list of all GOES files from specified path
        files = get_list_filenames(read_path, ".nc")
        return files

In [13]:
path = '/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/'

In [15]:
files_ = goes_files(path)
unique_times = list(set(map(parse_goes16_dates_from_file, files_)))

In [16]:
unique_times

['20202761419',
 '20182741420',
 '20202761439',
 '20182751411',
 '20202751409',
 '20182741405',
 '20202761409']

In [25]:
# get files from unique times
files = []
for ut in unique_times:
    files.extend(filter(lambda x: ut in x, files_))

In [32]:
files = list(filter(lambda x: unique_times[1] in x, files_))

In [35]:
files = list(filter(lambda x: "Rad" in x, files))

In [36]:
files

['/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/L1b/noaa-goes16/ABI-L1b-RadF/2018/274/14/OR_ABI-L1b-RadF-M4C01_G16_s20182741415224_e20182741420026_c20182741420080.nc',
 '/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/L1b/noaa-goes16/ABI-L1b-RadF/2018/274/14/OR_ABI-L1b-RadF-M4C02_G16_s20182741415224_e20182741420026_c20182741420075.nc',
 '/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/L1b/noaa-goes16/ABI-L1b-RadF/2018/274/14/OR_ABI-L1b-RadF-M4C03_G16_s20182741415224_e20182741420026_c20182741420081.nc',
 '/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/L1b/noaa-goes16/ABI-L1b-RadF/2018/274/14/OR_ABI-L1b-RadF-M4C04_G16_s20182741415224_e20182741420027_c20182741420080.nc',
 '/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/L1b/noaa-goes16/ABI-L1b-RadF/2018/274/14/OR_ABI-L1b-RadF-M4C05_G16_s20182741415224_e20182741420026_c20182741420082.nc',
 '/Users/anna.jungbluth/Desktop/git/rs_tools/data/goes16/L1b/noaa-goes16/ABI-L1b-RadF/2018/274/14/OR_ABI-L1b-RadF-M

In [37]:
ds.Rad.attrs

{'long_name': 'ABI L1b Radiances',
 'standard_name': 'toa_outgoing_radiance_per_unit_wavelength',
 'sensor_band_bit_depth': 10,
 'valid_range': array([   0, 1022], dtype=int16),
 'units': 'W m-2 sr-1 um-1',
 'resolution': 'y: 0.000028 rad x: 0.000028 rad',
 'grid_mapping': 'goes_imager_projection',
 'cell_methods': 't: point area: point',
 'ancillary_variables': 'DQF'}